In [1]:
import pandas as pd
from geopy.distance import vincenty
import geohash
import numpy as np
import csv
import operator
import collections
import datetime
from datetime import datetime
from collections import defaultdict
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
import gmplot

In [2]:
df = pd.read_csv('masterfile.csv')
df['ts'] = pd.to_datetime(df.time)

In [3]:
df.sort(columns=['ID'], inplace=True)
names=df['ID'].unique().tolist()

people = {}
for n in names:
    select = df.loc[df['ID']==n]
    people[n] = len(select['Day'].unique())

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


In [4]:
def round_to_5min(t):
    delta = pd.datetools.timedelta(minutes=t.minute%5, 
                               seconds=t.second, 
                               microseconds=t.microsecond)
    t -= delta
    if delta >= pd.datetools.timedelta(minutes=2.5):
        t += pd.datetools.timedelta(minutes=5)
    return t

In [5]:
df['ts10'] = df['ts'].apply(lambda x: round_to_5min(x))

In [6]:
drop_ids = [p for p in people if people[p] <= 5]
df = df.loc[~(df['ID'].isin(drop_ids))]

In [7]:
trial = df[['ts10','ID','lat','lon','accuracy']].drop_duplicates(subset=['ts10','ID'])

In [8]:
piv = pd.pivot_table(trial, index='ts10', columns='ID' ,values = ['lat','lon','accuracy'])

In [9]:
piv = piv.asfreq('5Min', method=None)

In [10]:
piv.groupby(pd.TimeGrouper('D')).count().tail()

lat                                                 \
ID         1rishabhtrivedi adithyapsv anshumanagrwl anuj7chauhan arch.b80   
ts10                                                                        
2016-04-30             288        181             0          189        4   
2016-05-01             253        285             0          237       16   
2016-05-02               0        208             0          208       22   
2016-05-03               0        277             0          189       26   
2016-05-04               0        205             0          250        2   

                                                                          \
ID         arkg1996 bhagyeshvikani bhardwaj.rish coolsush89 desai.deshna   
ts10                                                                       
2016-04-30        0            288           288        253            0   
2016-05-01        2            288           277        288          163   
2016-05-02        0            224            35        288          288   
2016-05-03        2            288           288        202          288   
2016-05-04        0            277           206        284          277   

             ...         accuracy                                             \
ID           ...    rudra.chandak sacheendra.t sagarparikh31 samriddhisimlai   
ts10         ...                                                               
2016-04-30   ...              285            0             1               0   
2016-05-01   ...              287            0             0               0   
2016-05-02   ...              280            0             0               0   
2016-05-03   ...              280            0             0               0   
2016-05-04   ...              282            0             0               0   

                                                                 \
ID         saumyadoshi shaleen.k.gupta tany.dudett umangjparmar   
ts10                                                              
2016-04-30         282               0         286          231   
2016-05-01         229               0         242          208   
2016-05-02         207               0          48          246   
2016-05-03         275               0         272          232   
2016-05-04         167               0         235          119   

                                     
ID         vaibhav29.07.97 vraj2026  
ts10                                 
2016-04-30             143      240  
2016-05-01             254      146  
2016-05-02             114      149  
2016-05-03               0      210  
2016-05-04               0       33  

[5 rows x 138 columns]

In [11]:
import itertools
ts = piv.index.tolist()

m = [list(g) for k, g in itertools.groupby(ts, key=lambda d: d.date())]
dates = list(set(map(lambda t: t.date(),ts)))

In [12]:
select = df.loc[df['ts10']=='2016-04-20 02:00:00']
print len(select)
print m[50][0]

116
2016-03-15 00:00:00


In [13]:
for i in range(50,len(m)):
    select = df.loc[df['ts10']=='2016-04-20 02:00:00']
    data_ = select[['lat','lon']]
    data = data_.as_matrix()
    kms_per_radian = 6371.0088
    epsilon = 0.02 / kms_per_radian
    db = DBSCAN(eps=epsilon, min_samples=10, algorithm='ball_tree', metric='haversine').fit(np.radians(data))
    cluster_labels = db.labels_
    num_clusters = len(set(cluster_labels))
    clusters = pd.Series([data[cluster_labels == n] for n in range(num_clusters)])
    print('Number of clusters: {}'.format(num_clusters))
    break

Number of clusters: 5


In [14]:
print clusters[3]


[[ 23.190571  72.628358]
 [ 23.19051   72.628306]
 [ 23.19051   72.628268]
 [ 23.190514  72.62822 ]
 [ 23.190514  72.62822 ]
 [ 23.190474  72.628265]
 [ 23.190531  72.62819 ]
 [ 23.190564  72.628227]
 [ 23.190546  72.628195]
 [ 23.190534  72.628196]
 [ 23.190535  72.628198]
 [ 23.190537  72.628201]
 [ 23.190543  72.628197]
 [ 23.19053   72.628188]]


In [81]:
'''
interact inputs: timestamp, epsilon (in Kms |float), min_samples (int)
interact output: plot

eps : float, optional

    The maximum distance between two samples for them to be considered as in the same neighborhood.

min_samples : int, optional

    The number of samples (or total weight) in a neighborhood for a point to be considered as a core point. This includes the point itself.

'''
def plot(ts,min_s,eps):
    select = df.loc[df['ts10']==ts]
    data_ = select[['lat','lon']]
    data = data_.as_matrix()
    kms_per_radian = 6371.0088
    epsilon = eps / kms_per_radian
    db = DBSCAN(eps=epsilon, min_samples=min_s, algorithm='ball_tree', metric='haversine').fit(np.radians(data))
    cluster_labels = db.labels_
    num_clusters = len(set(cluster_labels))
    clusters = pd.Series([data[cluster_labels == n] for n in range(num_clusters)])
    print('Number of clusters: {}'.format(num_clusters))
    colors= ['#FF0000','cornflowerblue','#FFFF00','#E9967A','#800000',
            '#ADD8E6','#00FF00','#FFA500','#C0C0C0','#C6DEFF',
            '#99C68E','#99C68E','#9DC209','#9DC209','#9DC209',
            '#ECE5B6','#ECE5B6','#E2A76F','#E2A76F','#A3FFCB',
            '#A3FFCB','#6F4E37','#E55451','#B38481','#E8ADAA',
            '#E0B0FF','#8467D7','#B8D667','#D66793','#908921','#21288F']
    gmap = gmplot.GoogleMapPlotter(23.188292, 72.628430,17)
    for i in range(0,len(clusters)):
        gmap.plot(clusters[i][:,0],clusters[i][:,1],c=colors[i],edge_width=10)
    gmap.scatter(data_['lat'], data_['lon'], color='k', size=2,marker=False)#,fill_color='fill', line_color="black",marker=False)
    gmap.draw("mymap.html")

In [82]:
plot('2016-04-20 9:00:00',1,0.02)

Number of clusters: 23


In [27]:
'''colors= ['#FF0000','cornflowerblue','#FFFF00','#E9967A','#800000']
gmap = gmplot.GoogleMapPlotter(23.188292, 72.628430,17)
for i in range(0,len(clusters)):
    gmap.plot(clusters[i][:,0],clusters[i][:,1],colors[i],edge_width=10)
gmap.scatter(data_['lat'], data_['lon'], '#3B0B39', size=1, marker=False)
gmap.draw("mymap.html")'''

In [ ]:
# ToDo: google maps plots for ipynbs: https://github.com/pbugnion/gmaps